In [ ]:
# Install BigQuery client library (run this cell once)
%pip install google-cloud-bigquery --quiet
%pip install bigquery-magics --quiet
%pip install db-dtypes --quiet
# Import libraries
from google.cloud import bigquery
import pandas as pd
%load_ext bigquery_magics
import db_dtypes
import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "salesforce-465614-2cf9e37da64b.json"
from google.cloud import bigquery
# Initialize BigQuery client
client = bigquery.Client()

<h2>Reporting Layer</h2>

In [ ]:
%%bqsql
create or replace view salesforce-465614.cust_analytics.report_customer_transactions_view as
select transaction_id,fact.customer_id,fact.product_id,fact.store_id,transaction_date,quantity,discount_applied,total_amount,
channel,payment_method,first_name,last_name,gender,age,signup_date,loyalty_program,email,city,state,country,store_type,store_location,
region,date_id,day_of_week,week_of_year,month,year,is_weekend from 
salesforce-465614.cust_analytics.fact_transactions fact
left join 
salesforce-465614.cust_analytics.dim_customers cust
on fact.customer_id=cust.customer_id
left join 
salesforce-465614.cust_analytics.dim_products product 
on fact.product_id=product.product_id
left join 
salesforce-465614.cust_analytics.stage_stores stores
on fact.store_id=stores.store_id
left join
salesforce-465614.cust_analytics.dim_date date
on fact.transaction_date=date.date_id
